# Manoeuvring parameter identification

Can the method that was introduced in the [Ball drop example](00.01_intro.ipynb#Example) be applied to a more complex system? This will be attempted in this chapter based on the results from the manoeuvring simulation from the [previous chapter](01.01_manoeuvring_simulation.ipynb#Simulate-data).

In [ ]:
# %load imports.py
## Local packages:

%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False  ## (To fix autocomplete)

## External packages:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

import numpy as np
import os
import plotly.express as px 
import plotly.graph_objects as go

import seaborn as sns

import matplotlib.pyplot as plt
if os.name == 'nt':
    plt.style.use('../docs/book/book.mplstyle')  # Windows

import sympy as sp
from sympy.physics.mechanics import (dynamicsymbols, ReferenceFrame,
                                      Particle, Point)
from sympy.physics.vector.printing import vpprint, vlatex
from IPython.display import display, Math, Latex, Markdown
from vessel_manoeuvring_models.substitute_dynamic_symbols import run, lambdify

import pyro

import sklearn
import pykalman
from statsmodels.sandbox.regression.predstd import wls_prediction_std
import statsmodels.api as sm

from scipy.integrate import solve_ivp

## Local packages:
from vessel_manoeuvring_models.data import mdl

from vessel_manoeuvring_models.symbols import *
from vessel_manoeuvring_models.parameters import *
import vessel_manoeuvring_models.symbols as symbols
from vessel_manoeuvring_models import prime_system
from vessel_manoeuvring_models.models import regression
from vessel_manoeuvring_models.visualization.regression import show_pred
from vessel_manoeuvring_models.visualization.plot import track_plot
from vessel_manoeuvring_models.visualization.equation import Equation
from myst_nb import glue

## Load models:
# (Uncomment these for faster loading):
#import vessel_manoeuvring_models.models.vmm_nonlinear_EOM  as vmm 
import vessel_manoeuvring_models.models.vmm_martin  as vmm

## Examples
from docs.book.example_1 import ship_parameters, df_parameters, ps, ship_parameters_prime

## Decoupling
The hydrodynamic derivatives in a mathematical model for ship manoeuvring should be identified by expressing the ODE as a OLS regression. There is a coupling between the sway and yaw equation in this model. These equations need to be decoupled {cite:p}`wang_parameter_2021`, in a similar way as how it was done in the previous chapter. The simulation model can be expressed as:

In [ ]:
A = vmm.simulator.A
b = vmm.simulator.b
acceleration = sp.matrices.MutableDenseMatrix([u1d,v1d,r1d])
eq_simulator = sp.Eq(sp.UnevaluatedExpr(A)*sp.UnevaluatedExpr(acceleration),sp.UnevaluatedExpr(b))
Math(vlatex(eq_simulator))

$X_{qs}$, $Y_{qs}$, $N_{qs}$ are the quasi static hydrodynamic force models, that contains the hydrodynamic derivatives ($Y_{uv}$ etc.) that should be identified.

The PIT should be expressed as three regressions based on $\dot{u}$, $\dot{v}$ and $\dot{r}$:

In [ ]:
coeff_matrix = sp.matrices.MutableDenseMatrix([A_coeff*X_X,B_coeff*X_Y,C_coeff*X_N])
eq_regression = sp.Eq(acceleration, coeff_matrix)
Math(vlatex(eq_regression))

Where $A_{coeff}$, $B_{coeff}$ and $C_{coeff}$ are coefficient vectors from where the hydrodynamic derivatives can be obtained with decoupling as follows, where eq.{eq}`eq_coeff` has been inserted into eq.{eq}`eq_simulator`:

In [ ]:
Math(vlatex(sp.Eq(sp.UnevaluatedExpr(A)*sp.UnevaluatedExpr(coeff_matrix),sp.UnevaluatedExpr(b))))

```{glue:math} sym_eq
:label: eq_simulator_coeff
```

And the right hand side of this equation can also be expressed with vectors:

In [ ]:
X_qs_, Y_qs_, N_qs_ = sp.symbols('X_qs, Y_qs, N_qs')
eq_X_qs = sp.Eq(X_qs_*X_X, b[0])
Math(vlatex(eq_X_qs))

In [ ]:
eq_Y_qs = sp.Eq(X_qs_*X_Y, b[1])
Math(vlatex(eq_Y_qs))

In [ ]:
eq_N_qs = sp.Eq(N_qs_*X_N, b[2])
Math(vlatex(eq_N_qs))

So that the parameter vectors $X_{qs}$, $Y_{qs}$, $N_{qs}$ can be calculated from $A_{coeff}$, $B_{coeff}$ and $C_{coeff}$ in the following way.

In [ ]:
Math(vlatex(sp.Eq(sp.UnevaluatedExpr(A)*sp.UnevaluatedExpr(sp.matrices.MutableDenseMatrix([A_coeff,B_coeff,C_coeff])),
                  sp.UnevaluatedExpr(sp.matrices.MutableDenseMatrix([X_qs_,Y_qs_,N_qs_])))))

## Regression
The parameter vectors : $A_{coeff}$, $B_{coeff}$ and $C_{coeff}$ are determined with OLS regression:

### Load simulation results:

In [ ]:
df_result = pd.read_csv('test.csv', index_col=0)  # (gnereated by:01.01_manoeuvring_simulation.ipynb)
df_result['U'] = np.sqrt(df_result['u']**2 + df_result['v']**2)

#df_result = df_result.iloc[100:-100]

df = ps.prime(df_result, U=df_result['U'])

In [ ]:
df_result.head()

### N

In [ ]:
N_ = sp.symbols('N_')

diff_eq_N = regression.DiffEqToMatrix(ode=vmm.N_qs_eq.subs(N_D,N_), 
                                      label=N_, base_features=[delta,u,v,r])

In [ ]:
X = diff_eq_N.calculate_features(data=df)
y = diff_eq_N.calculate_label(y=df['r1d'])

model_N = sm.OLS(y,X)
results_N = model_N.fit()

show_pred(X=X,y=y,results=results_N, label=r'$\dot{r}$')

### Y

In [ ]:
Y_ = sp.symbols('Y_')
diff_eq_Y = regression.DiffEqToMatrix(ode=vmm.Y_qs_eq.subs(Y_D,Y_), 
                                      label=Y_, base_features=[delta,u,v,r])

In [ ]:
X = diff_eq_Y.calculate_features(data=df)
y = diff_eq_Y.calculate_label(y=df['v1d'])


model_Y = sm.OLS(y,X)
results_Y = model_Y.fit()

show_pred(X=X,y=y,results=results_Y, label=r'$\dot{v}$')

### X

In [ ]:
X_ = sp.symbols('X_')
diff_eq_X = regression.DiffEqToMatrix(ode=vmm.X_qs_eq.subs(X_D,X_), 
                                      label=X_, base_features=[delta,u,v,r,thrust])

In [ ]:
X = diff_eq_X.calculate_features(data=df)
y = diff_eq_X.calculate_label(y=df['u1d'])

model_X = sm.OLS(y,X)
results_X = model_X.fit()

show_pred(X=X,y=y,results=results_X, label=r'$\dot{u}}$')

### Decoupling

In [ ]:
subs = {value:key for key,value in p.items()}
A_ = A*sp.matrices.MutableDenseMatrix([A_coeff,B_coeff,C_coeff])
A_lambda=lambdify(A_.subs(subs))

In [ ]:
results_summary_X = regression.results_summary_to_dataframe(results_X)
results_summary_Y = regression.results_summary_to_dataframe(results_Y)
results_summary_N = regression.results_summary_to_dataframe(results_N)

A_coeff_ = results_summary_X['coeff']
B_coeff_ = results_summary_Y['coeff']
C_coeff_ = results_summary_N['coeff']

coeffs = run(A_lambda,A_coeff=A_coeff_.values, B_coeff=B_coeff_.values, C_coeff=C_coeff_.values, 
    **df_parameters['prime'], **ship_parameters_prime)

The way that the regression is formulated, inertial forces, such as centrifugal force will be included into the derivatives (I think) which means that centrifugal force : $-m \cdot r \cdot u$ will be included into $Y_{ur}$ coefficient. This coefficient is therefore not pure hydrodynamic, and can potentially be counted twice..?
The coefficients are recalculated below to avooid this:

In [ ]:
results_summary_X['decoupled'] = coeffs[0][0]
results_summary_Y['decoupled'] = coeffs[1][0]
results_summary_N['decoupled'] = coeffs[2][0]

x_G_ = ship_parameters_prime['x_G']
m_ = ship_parameters_prime['m']

results_summary_X.loc['Xrr','decoupled']+=(-m_*x_G_)
results_summary_X.loc['Xvr','decoupled']+=(-m_)
results_summary_Y.loc['Yur','decoupled']+=m_
results_summary_N.loc['Nur','decoupled']+=m_*x_G_

### Add the regressed parameters
Hydrodynamic derivatives that depend on acceleration cannot be obtained from the VCT regression. They are however essential if a time simulation should be conducted. These values have then been taken from Brix semi empirical formulas for the simulations below.

In [ ]:
df_parameters_all = df_parameters.copy()
for other in [results_summary_X, results_summary_Y, results_summary_N]:
    df_parameters_all = df_parameters_all.combine_first(other)

df_parameters_all.rename(columns={'decoupled':'regressed'}, inplace=True)
df_parameters_all.drop(columns=['brix_lambda'], inplace=True)

df_parameters_all['regressed'] = df_parameters_all['regressed'].combine_first(df_parameters_all['prime'])  # prefer regressed
#df_parameters_all['regressed'].fillna(0,inplace=True)

# Simulation

A simulation with the regressed hydrodynamic coefficients can now be conducted. The figures below compare results from these simulation and the original data.

In [ ]:
parameters=df_parameters_all['regressed'].copy()

df_ = df_result.copy()

d_psi = np.deg2rad(-0.41)
df_['x0'] = df_['x0']*np.cos(d_psi) - df_['y0']*np.sin(d_psi)
df_['y0'] = df_['x0']*np.sin(d_psi) + df_['y0']*np.cos(d_psi)
df_['u'] = df_['u']*np.cos(d_psi) - df_['v']*np.sin(d_psi)
df_['v'] = df_['u']*np.sin(d_psi) + df_['v']*np.cos(d_psi)
df_['psi']+=d_psi

result_regression = vmm.simulator.simulate(df_=df_, 
                                           parameters=parameters, 
                                           ship_parameters=ship_parameters, 
                                           control_keys=['delta','thrust'], 
                                           primed_parameters=True,
                                           prime_system=ps,
                                           name='regressed')

result_regression.df_model_test=df_result  # dirty

In [ ]:
result_regression.track_plot(compare=True);

In [ ]:
result_regression.plot(compare=True);

In [ ]:
r2s = result_regression.score()
r2s.plot(kind='bar');

In [ ]:
fig,ax=plt.subplots()
key='r1d'
result_regression.df_model_test.plot(y=key, ax=ax)
result_regression.result.plot(y=key, ax=ax)

In [ ]:
from vessel_manoeuvring_models.models.regression import MotionRegression

In [ ]:
df_parameters.head()

In [ ]:
added_masses = df_parameters.groupby(by='state').get_group('dot')['prime'].dropna().to_dict()
added_masses

In [ ]:

regression = MotionRegression(vmm=vmm, data=df, added_masses=added_masses, ship_parameters=ship_parameters, prime_system=ps, 
                              base_features=[u,v,r,delta,thrust])

In [ ]:
regression.show()

In [ ]:
model = regression.create_model(control_keys=['delta','thrust'])

In [ ]:
result_regression = model.simulate(df_=df_)

In [ ]:
result_regression.track_plot(compare=True);

In [ ]:
result_regression.plot(compare=True);